In [1]:
#导入数据

#hyperparameter

#input placeholder

#define weight and bias

#define rnn

#prediction

#cost

#train op

# trainning



In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# set random seed for comparing the two result calculations
tf.set_random_seed(1)

# this is data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/james/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size 

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#hyperparameter
batch_size = 28 # once study 20 pictures
input_size = 28 #image28*28
output_size = 10 #output 10 class
state_size = 128 #number of neurons
time_step = 28 # 1 image need to learn 28 step
lr = 0.001
training_iteration  =100000



In [4]:
#input placeholder

x = tf.placeholder(tf.float32,[None,time_step,input_size])
y = tf.placeholder(tf.float32,[None,output_size])

    

In [5]:
#define weight and bias
Weights = {
    'in': tf.Variable(tf.random_normal([input_size,state_size])),
    'out': tf.Variable(tf.random_normal([state_size,output_size]))
}
bias = {
    'in': tf.Variable(tf.constant(0.1,shape=[state_size])),
     'out':tf.Variable(tf.constant(0.1,shape=[output_size]))
}

#define rnn only one layer neuron
def RNN(X,Weights, bias):
    #define input layer
    #reshape
    X = tf.reshape(X,[-1,input_size])
    
    x_in = tf.matmul(X,Weights['in'])+bias['in']
    
    x_in = tf.reshape(x_in,[-1,time_step,state_size])
    
    #define neutron layer
    
    
    #cell = tf.contrib.rnn.BasicLSTMCell(state_size)
   
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        cell = tf.nn.rnn_cell.BasicLSTMCell(state_size, forget_bias=1.0, state_is_tuple=True)
    else:
        cell = tf.contrib.rnn.BasicLSTMCell(state_size)

    init_state = cell.zero_state(batch_size,dtype=tf.float32)
    
    
    outputs, final_state = tf.nn.dynamic_rnn(cell,x_in,initial_state=init_state, time_major=True)
    
   
   
    tf.reset_default_graph()
    
    #define output layer
    #results = tf.matmul(final_state[1],Weights['out'])+bias['out']
    
    
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        outputs = tf.unpack(tf.transpose(outputs, [1, 0, 2]))    # states is the last outputs
    else:
        outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul(outputs[-1], Weights['out']) + bias['out']    # shape = (128, 10)
    return results
    

In [6]:
def RNN(X, weights, biases):
    # hidden layer for input to cell
    ########################################

    # transpose the inputs shape from
    # X ==> (128 batch * 28 steps, 28 inputs)
    X = tf.reshape(X, [-1, input_size])

    # into hidden
    # X_in = (128 batch * 28 steps, 128 hidden)
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batch, 28 steps, 128 hidden)
    X_in = tf.reshape(X_in, [-1, time_step, state_size])

    # cell
    ##########################################

    # basic LSTM Cell.
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        cell = tf.nn.rnn_cell.BasicLSTMCell(state_size, forget_bias=1.0, state_is_tuple=True)
    else:
        cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    # lstm cell is divided into two parts (c_state, h_state)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)

    # You have 2 options for following step.
    # 1: tf.nn.rnn(cell, inputs);
    # 2: tf.nn.dynamic_rnn(cell, inputs).
    # If use option 1, you have to modified the shape of X_in, go and check out this:
    # https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py
    # In here, we go for option 2.
    # dynamic_rnn receive Tensor (batch, steps, inputs) or (steps, batch, inputs) as X_in.
    # Make sure the time_major is changed accordingly.
    outputs, final_state = tf.nn.dynamic_rnn(cell, X_in, initial_state=init_state, time_major=False)

    # hidden layer for output as the final results
    #############################################
    # results = tf.matmul(final_state[1], weights['out']) + biases['out']

    # # or
    # unpack to list [(batch, outputs)..] * steps
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        outputs = tf.unpack(tf.transpose(outputs, [1, 0, 2]))    # states is the last outputs
    else:
        outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']    # shape = (128, 10)

    return results

In [7]:
#prediction
pred = RNN(x,Weights,bias)
#classification use cross entropy
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))

#training
#hwo to train
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    step =0
    while step*batch_size < training_iteration:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size,time_step,input_size])
        sess.run(train_op,feed_dict={x:batch_xs,y:batch_ys})
        if step % 20 == 0:
            print("cost:")
            print(sess.run(cost,feed_dict={x:batch_xs,y:batch_ys}))
            print(sess.run(accuracy,feed_dict={x:batch_xs,y:batch_ys}))
        step+=1

cost:
2.0768225
0.25
cost:
1.822468
0.32142857
cost:
1.0405341
0.60714287
cost:
0.6364149
0.78571427
cost:
1.0791943
0.53571427
cost:
0.75657755
0.78571427
cost:
0.45082822
0.85714287
cost:
0.39067647
0.9285714
cost:
0.48475432
0.85714287
cost:
0.24113396
0.9285714
cost:
0.7184387
0.75
cost:
0.38351884
0.8214286
cost:
0.40306586
0.85714287
cost:
0.58496076
0.85714287
cost:
0.09058191
1.0
cost:
0.34821385
0.8214286
cost:
0.25940198
0.89285713
cost:
0.20817201
0.89285713
cost:
0.3211732
0.9285714
cost:
0.41746798
0.85714287
cost:
0.4849966
0.89285713
cost:
0.45463863
0.85714287
cost:
0.2055241
0.9285714
cost:
0.16562404
0.96428573
cost:
0.39999375
0.8214286
cost:
0.23201199
0.9285714
cost:
0.2948124
0.85714287
cost:
0.21413346
0.9285714
cost:
0.09519992
0.96428573
cost:
0.34091282
0.89285713
cost:
0.15474795
0.9285714
cost:
0.24461709
0.96428573
cost:
0.09459279
0.96428573
cost:
0.15565677
0.96428573
cost:
0.37851065
0.89285713
cost:
0.332082
0.89285713
cost:
0.096121244
1.0
cost:
0.1153